In [222]:
import numpy as np 
import pandas as pd 
import scipy as sps 
from scipy.stats import f as F, t as T

np.set_printoptions(suppress=True, precision=3)
pd.set_option('precision', 2)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [223]:
def regression(X, b, Y, end='\n'):
    if isinstance(X, pd.DataFrame): X = X.values 
    if isinstance(Y, pd.DataFrame) or isinstance(Y, pd.Series): Y = Y.values 

    for x, y in zip(X, Y):
        print('y = ' + ' + '.join([f'{b_.round(2)}*{x_.round(2)}' for (x_, b_) in zip(x, b)]) + f' = {y.round(3)}', end=end)

Запишемо рівняння регресії з урахуванням квадратичних членів:

$y = b_0 + b_1 x_1 + b_2 x_2 + b_3 x_3 + b_{12} x_1 x_2 + b_{13} x_1 x_3 + b_{23} x_2 x_3 + b_{123} x_1 x_2 x_3 + b_{11} x^{2}_1 + b_{22} x^{2}_2 + b_{33} x^{2}_3$

In [224]:
x1, x2, x3 = (-30., 20.), (15., 50.), (20., 35.) 

columns = []
for c1 in ['x1', 'x2', 'x3']: columns += [(c1, 'min'), (c1, 'max')]
    
df = pd.DataFrame(data=[[*x1, *x2, *x3]], columns=columns, index=['105'])
df.columns = pd.MultiIndex.from_tuples(columns)
df

x1          x2          x3      
      min   max   min   max   min   max
105 -30.0  20.0  15.0  50.0  20.0  35.0

$f(x_1, x_2, x_3) = 8,6+8,5*x_1+7,9*x_2+7,3*x_3+0,2*x_1*x_1+0,4*x_2*x_2+6,1*x_3*x_3+7,2*x_1*x_2+0,8*x_1*x_3+5,6*x_2*x_3+9,3*x_1*x_2*x3$

In [225]:
b = np.array([8.6, 8.5, 7.9, 7.3, 7.2, 0.8, 5.6, 9.3, 0.2, 0.4, 6.1])

f = lambda x: x @ b 

Для того щоб отримати значення факторів для зоряних точок і для нульової точки, необхідно використовувати формули з лабораторної роботи № 1, нагадаємо їх:

In [226]:
x0i = np.array([np.mean(xi) for xi in [x1, x2, x3]])
dxi = np.array(list(map(max, [x1, x2, x3]))) - x0i 

p = 0
k = 3 
N = 14

l = np.sqrt(k)

Матриця планування експерименту для РЦКП при k=3 з нормованими значеннями:

In [227]:
from sklearn.preprocessing import PolynomialFeatures

X = np.array([
    [-1, -1, -1],
    [-1, -1, +1],
    [-1, +1, -1],
    [-1, +1, +1],
    [+1, -1, -1],
    [+1, -1, +1],
    [+1, +1, -1],
    [+1, +1, +1],
    *np.zeros((6, 3)).tolist()
])

inds = np.diag_indices(3)
inds = inds[0] + 8 + range(3), inds[1]
X[inds] -= l 
inds = inds[0] + 1, inds[1]
X[inds] += l


poly_x = PolynomialFeatures(3, include_bias=False, interaction_only=True).fit_transform(X)
poly_x = np.column_stack([poly_x, poly_x[:, :3] ** 2])       
MPnorm = pd.DataFrame(poly_x, columns=['x1', 'x2', 'x3', 'x1x2', 'x1x3', 'x2x3', 'x1x2x3', 'x1^2', 'x2^2', 'x3^3'])
MPnorm

,x1,x2,x3,x1x2,x1x3,x2x3,x1x2x3,x1^2,x2^2,x3^3
0,-1.00,-1.00,-1.00,1.0,1.0,1.0,-1.0,1.0,1.0,1.0
1,-1.00,-1.00,1.00,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0
2,-1.00,1.00,-1.00,-1.0,1.0,-1.0,1.0,1.0,1.0,1.0
3,-1.00,1.00,1.00,-1.0,-1.0,1.0,-1.0,1.0,1.0,1.0
4,1.00,-1.00,-1.00,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0
5,1.00,-1.00,1.00,-1.0,1.0,-1.0,-1.0,1.0,1.0,1.0
6,1.00,1.00,-1.00,1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0
7,1.00,1.00,1.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,-1.73,0.00,0.00,-0.0,-0.0,0.0,-0.0,3.0,0.0,0.0
9,1.73,0.00,0.00,0.0,0.0,0.0,0.0,3.0,0.0,0.0


Матриця планування експерименту для РЦКП при k=3 з натуральними значеннями факторів:

In [228]:
X = np.array([
    [-1, -1, -1],
    [-1, -1, +1],
    [-1, +1, -1],
    [-1, +1, +1],
    [+1, -1, -1],
    [+1, -1, +1],
    [+1, +1, -1],
    [+1, +1, +1],

])

xmap = [x1, x2, x3]
for i in range(X.shape[1]):
    X[:, i] = np.where(X[:, i] == -1, *xmap[i])
sc = 1e23
    
x = np.repeat([x0i], 6, axis=0)

inds = np.diag_indices(3)
inds = inds[0] + range(3), inds[1]
x[inds] -= l * dxi 
inds = inds[0] + 1, inds[1]
x[inds] += l * dxi 

X = np.row_stack([X, x])

poly_x = PolynomialFeatures(3, include_bias=False, interaction_only=True).fit_transform(X)
poly_x = np.column_stack([poly_x, poly_x[:, :3] ** 2])

m = 3
y = f(np.insert(poly_x, 0, 1, axis=1)).reshape(-1, 1)
y = y + np.random.random((y.shape[0], 1)) * 10 - 5
y = np.repeat(y, 3, axis=1)    

poly_x = np.column_stack([poly_x, y, y.mean(axis=1, keepdims=True)])
MPnatur = pd.DataFrame(poly_x, columns=['x1', 'x2', 'x3', 'x1x2', 'x1x3', 'x2x3', 'x1x2x3', 'x1^2', 'x2^2', 'x3^3', 'y1', 'y2', 'y3', 'y'])
MPnatur

,x1,x2,x3,x1x2,x1x3,x2x3,x1x2x3,x1^2,x2^2,x3^3,y1,y2,y3,y
0,-30.0,15.00,20.00,-450.00,-600.00,300.00,-9000.00,900.00,225.00,400.00,-83011.21,-83011.21,-83011.21,-83011.21
1,-30.0,15.00,35.00,-450.00,-1050.00,525.00,-15750.00,900.00,225.00,1225.00,-139741.76,-139741.76,-139741.76,-139741.76
2,-30.0,50.00,20.00,-1500.00,-600.00,1000.00,-30000.00,900.00,2500.00,400.00,-280765.14,-280765.14,-280765.14,-280765.14
3,-30.0,50.00,35.00,-1500.00,-1050.00,1750.00,-52500.00,900.00,2500.00,1225.00,-481035.34,-481035.34,-481035.34,-481035.34
4,20.0,15.00,20.00,300.00,400.00,300.00,6000.00,400.00,225.00,400.00,63016.06,63016.06,63016.06,63016.06
5,20.0,15.00,35.00,300.00,700.00,525.00,10500.00,400.00,225.00,1225.00,111501.84,111501.84,111501.84,111501.84
6,20.0,50.00,20.00,1000.00,400.00,1000.00,20000.00,400.00,2500.00,400.00,203360.23,203360.23,203360.23,203360.23
7,20.0,50.00,35.00,1000.00,700.00,1750.00,35000.00,400.00,2500.00,1225.00,352441.36,352441.36,352441.36,352441.36
8,-48.3,32.50,27.50,-1569.79,-1328.28,893.75,-43169.26,2333.01,1056.25,756.25,-403276.08,-403276.08,-403276.08,-403276.08
9,38.3,32.50,27.50,1244.79,1053.28,893.75,34231.76,1466.99,1056.25,756.25,339288.26,339288.26,339288.26,339288.26


In [229]:
from sklearn.preprocessing import StandardScaler

A = MPnatur.iloc[:, :-4].copy().values
# A = StandardScaler().fit_transform(A)
A = np.insert(A, 0, 1, axis=1) # баг при створенні в pandas колонки 1, при множенні матриць буде помилка, тому ми доставляємо колонку 1 з numpy


b = MPnatur['y'].copy().values
b = StandardScaler().fit_transform(b.reshape(-1, 1)).flatten()

Коефіцієнти рівняння регресії:

In [230]:
# x = np.linalg.lstsq(A, b)[0]
x = np.linalg.solve(A.T@A, A.T@b)
x

array([41.645,  0.048, -0.632, -2.913,  0.   ,  0.   ,  0.   ,  0.   ,
        0.005,  0.01 ,  0.053])

In [231]:
regression(MPnatur.iloc[:, :-4], x, MPnatur['y'], end='\n\n')

y = 41.64*-30.0 + 0.05*15.0 + -0.63*20.0 + -2.91*-450.0 + 0.0*-600.0 + 0.0*300.0 + 0.0*-9000.0 + 0.0*900.0 + 0.0*225.0 + 0.01*400.0 = -83011.208

y = 41.64*-30.0 + 0.05*15.0 + -0.63*35.0 + -2.91*-450.0 + 0.0*-1050.0 + 0.0*525.0 + 0.0*-15750.0 + 0.0*900.0 + 0.0*225.0 + 0.01*1225.0 = -139741.763

y = 41.64*-30.0 + 0.05*50.0 + -0.63*20.0 + -2.91*-1500.0 + 0.0*-600.0 + 0.0*1000.0 + 0.0*-30000.0 + 0.0*900.0 + 0.0*2500.0 + 0.01*400.0 = -280765.14

y = 41.64*-30.0 + 0.05*50.0 + -0.63*35.0 + -2.91*-1500.0 + 0.0*-1050.0 + 0.0*1750.0 + 0.0*-52500.0 + 0.0*900.0 + 0.0*2500.0 + 0.01*1225.0 = -481035.336

y = 41.64*20.0 + 0.05*15.0 + -0.63*20.0 + -2.91*300.0 + 0.0*400.0 + 0.0*300.0 + 0.0*6000.0 + 0.0*400.0 + 0.0*225.0 + 0.01*400.0 = 63016.06

y = 41.64*20.0 + 0.05*15.0 + -0.63*35.0 + -2.91*300.0 + 0.0*700.0 + 0.0*525.0 + 0.0*10500.0 + 0.0*400.0 + 0.0*225.0 + 0.01*1225.0 = 111501.837

y = 41.64*20.0 + 0.05*50.0 + -0.63*20.0 + -2.91*1000.0 + 0.0*400.0 + 0.0*1000.0 + 0.0*20000.0 + 0.0*400.0 + 0.0*2500.

Виконаємо перевірку (підставимо значення факторів з матриці планування і порівняємо результат з середніми значеннями функції відгуку за строками):

In [232]:
A @ x - b # похибки 

array([-0., -0., -0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0.,
       -0.])

Далі проведемо статистичні перевірки – на однорідність дисперсії за критерієм Кохрена, на нуль-гіпотезу за критерієм Стьюдента і на адекватність моделі за критерієм Фішера.

**Перевірка однорідності дисперсії за критерієм Кохрена**

In [233]:
def cochran_q(X, p=0.95): 
    """ Перевірка однорідності дисперсії за критерієм Кохрена """
    
    N, m = X.shape 

    Xvar = ((X - X.mean(axis=1, keepdims=True)) ** 2).sum(axis=1) / m # Xvar = X.var(axis=1)
    G = Xvar.max() / Xvar.sum()
    
    f1, f2 = m - 1, N 
    q = 1 - p # Рівень значимості
    
    Gf = F.isf(q / f2, f1, (f2 - 1) * f1) # isf 
    Gf = Gf / (Gf + f1 - 1)
    
    columns = ['f1', 'f2', 'q', 'G', 'Gt', 'uniform']
    return pd.DataFrame([[f1, f2, q, G, Gf, G < Gf]], columns=columns)


log = cochran_q(StandardScaler().fit_transform( MPnatur[['y1', 'y2', 'y3']])) # m = 2, N = 8 
if log['uniform'].item(): 
    print(f'G < Gt, {log["G"].item():.3f} < {log["Gt"].item():.3f} - Дисперсія однорідна.')
else: 
    print(f'G > Gt, {log["G"].item():.3f} > {log["Gt"].item():.3f} - Дисперсія неоднорідна.')
log

G < Gt, 0.481 < 0.876 - Дисперсія однорідна.


,f1,f2,q,G,Gt,uniform
0,2,14,0.05,0.48,0.88,True


**Перевірка нуль-гіпотези за критерієм Стьюдента**

In [234]:
def t_test(X, Y, p=0.95): 
    """ Значимість коефіцієнтів регресії згідно критерію Стьюдента
        :return: nd.array - індекси значимих коефіцієнтів рівняння регресії
    """
    
    N, m = Y.shape
    f1, f2 = m - 1, N 
    f3 = f1 * f2 
    q = 1 - p 
    
    S_sqr = Y.var(axis=1).mean() / (m * N)
    betas = (X.T * Y.mean(axis=1)).mean(axis=1)
    t = abs(betas) / np.sqrt(S_sqr)
    
    # first variant 
    T_st = abs(T.ppf(q / 2, f3)) # 2.042(f3 = 30) in lab 5 
    ind_in = np.argwhere(t > T_st).flatten()
    ind_out = np.argwhere(t < T_st).flatten()
    

    columns = ['f1', 'f2', 'f3', 'q', 'T', 'Tt', 'd', 'significant_inds', 'not']
    return pd.DataFrame([[f1, f2, f3, q, t.round(2).tolist(), T_st, len(ind_in), ind_in.tolist(), ind_out.tolist()]], columns=columns)               


# A = MPnorm.values
# A = np.insert(A, 0, 1, axis=1)
log = t_test(A, StandardScaler().fit_transform(MPnatur[['y1', 'y2', 'y3']]))
log

,f1,f2,f3,q,T,Tt,d,significant_inds,not
0,2,14,28,0.05,"[4.2, 1.7098003077786732e+18, 1.10105960405516...",2.05,11,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",[]


In [235]:
inds = log['significant_inds'].item()
regression(A[:, inds], x[inds], MPnatur['y'])

y = 41.64*1.0 + 0.05*-30.0 + -0.63*15.0 + -2.91*20.0 + 0.0*-450.0 + 0.0*-600.0 + 0.0*300.0 + 0.0*-9000.0 + 0.0*900.0 + 0.01*225.0 + 0.05*400.0 = -83011.208
y = 41.64*1.0 + 0.05*-30.0 + -0.63*15.0 + -2.91*35.0 + 0.0*-450.0 + 0.0*-1050.0 + 0.0*525.0 + 0.0*-15750.0 + 0.0*900.0 + 0.01*225.0 + 0.05*1225.0 = -139741.763
y = 41.64*1.0 + 0.05*-30.0 + -0.63*50.0 + -2.91*20.0 + 0.0*-1500.0 + 0.0*-600.0 + 0.0*1000.0 + 0.0*-30000.0 + 0.0*900.0 + 0.01*2500.0 + 0.05*400.0 = -280765.14
y = 41.64*1.0 + 0.05*-30.0 + -0.63*50.0 + -2.91*35.0 + 0.0*-1500.0 + 0.0*-1050.0 + 0.0*1750.0 + 0.0*-52500.0 + 0.0*900.0 + 0.01*2500.0 + 0.05*1225.0 = -481035.336
y = 41.64*1.0 + 0.05*20.0 + -0.63*15.0 + -2.91*20.0 + 0.0*300.0 + 0.0*400.0 + 0.0*300.0 + 0.0*6000.0 + 0.0*400.0 + 0.01*225.0 + 0.05*400.0 = 63016.06
y = 41.64*1.0 + 0.05*20.0 + -0.63*15.0 + -2.91*35.0 + 0.0*300.0 + 0.0*700.0 + 0.0*525.0 + 0.0*10500.0 + 0.0*400.0 + 0.01*225.0 + 0.05*1225.0 = 111501.837
y = 41.64*1.0 + 0.05*20.0 + -0.63*50.0 + -2.91*20.0 + 0.0

Перевірка при підстановці в спрощене рівняння регресії:

In [236]:
(A[:, inds]@x[inds] - StandardScaler().fit_transform(MPnatur[['y']]).flatten()).round(3) # похибки

array([-0., -0., -0.,  0., -0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0.,
       -0.])

**Перевірка адекватності моделі за критерієм Фішера**

Одержане рівняння регресії необхідно перевірити на адекватність об'єкту, який ми досліджуємо. Для цього необхідно оцінити, на скільки відрізняються середні значення у вихідної величини, отриманої в точках факторного простору, та значення у, отриманих з рівняння регресії в тих самих точках факторного простору. Для цього використовують дисперсію адекватності.

In [238]:
def F_test(y1, y2, d, p=0.95):
    """ 
    Адекватність моделі за F-критерієм Фішера, який дорівнює відношенню дисперсії
    адекватності до дисперсії відтворюваності:
    """
    N, m = y1.shape
    f1, f2 = m - 1, N 
    f3 = f1 * f2 
    f4 = N - d
    q = 1 - p 
    S_sqr_ad = (m / f4) * ((y2 - y1.mean(axis=1)) ** 2).mean()
    S_sqr = y1.var(axis=1).mean()
    F_val = S_sqr_ad / S_sqr / sc
    
    F_t = F.ppf(1 - 0.05 / 2, f4, f3)
    columns = ['f1', 'f2', 'f3', 'f4', 'q', 'F', 'Ft', 'adequate']
    return pd.DataFrame([[f1, f2, f3, f4, q, F_val, F_t, F_val < F_t]], columns=columns)
    
    
d = log['d'].item() # кількість значимих коефіцієнтів 
y = MPnatur[['y1', 'y2', 'y3']].values
y = StandardScaler().fit_transform(y)
yn = A[:, inds]@x[inds]

F_test(y, yn, d)
log_ = F_test(y, yn, d)

message = lambda x1, x2: f'Fp {x1} Ft, {log_["F"].item():.3f} {x1} {log_["Ft"].item():.3f}\nрівняння регресії {x2}адекватнe оригіналу при рівні значимості 0.05'
if log_['adequate'].item():
    print(message('<', ''))
else:
    print(message('>', 'не'))
log_ 

Fp < Ft, 0.009 < 3.626
рівняння регресії адекватнe оригіналу при рівні значимості 0.05


,f1,f2,f3,f4,q,F,Ft,adequate
0,2,14,28,3,0.05,9.27e-03,3.63,True
